In [1]:
import numpy as np
import pandas as pd

In [2]:
import re
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

In [30]:
file_path = "C:\\Users\\adity\\Downloads\\kaggle1.txt"

with open(file_path, 'r', encoding='utf-8') as file:
    text_data = file.readlines()

print(text_data[:5])

['fraud\thello, i m bank manager of SBI, ur debit card is about to expire would u want to issue new  card.\n', 'fraud\tTodays Vodafone numbers ending with 4882 are selected to a receive a £350 award. If your number matches call 09064019014 to receive your £350 award.\n', "normal\tPlease don't say like that. Hi hi hi\n", 'normal\tThank you!\n', 'normal\tOh that was a forwarded message. I thought you send that to me.\n']


In [31]:
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)  
    text = text.lower() 
    return text

text_data_cleaned = [preprocess_text(text) for text in text_data]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data_cleaned)

sequences = tokenizer.texts_to_sequences(text_data_cleaned)

max_length = max(len(seq) for seq in sequences)  
X = pad_sequences(sequences, maxlen=max_length, padding='post')

print(X[:5])  


[[  18   98    3  436  193 1239   16 1118   50 1562  215    8   68    4
  3998  129   10   62    4 1375  104  215    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [  18  448  934  720  875   34   19  437    4    6  335    6  413   35
    11   86 1240   22    4  335   11  413

In [32]:
y = [1 if 'fraud' in text else 0 for text in text_data]  


In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")


Training data shape: (4745, 172)
Test data shape: (1187, 172)


In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, SpatialDropout1D

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_length))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))


Epoch 1/5
75/75 ━━━━━━━━━━━━━━━━━━━━ 27s 284ms/step - accuracy: 0.8450 - loss: 0.4065 - val_accuracy: 0.8905 - val_loss: 0.3456
Epoch 2/5
75/75 ━━━━━━━━━━━━━━━━━━━━ 22s 287ms/step - accuracy: 0.8975 - loss: 0.3353 - val_accuracy: 0.8905 - val_loss: 0.3473
Epoch 3/5
75/75 ━━━━━━━━━━━━━━━━━━━━ 21s 286ms/step - accuracy: 0.9026 - loss: 0.3191 - val_accuracy: 0.8905 - val_loss: 0.3457
Epoch 4/5
75/75 ━━━━━━━━━━━━━━━━━━━━ 22s 299ms/step - accuracy: 0.8962 - loss: 0.3343 - val_accuracy: 0.8905 - val_loss: 0.3458
Epoch 5/5
75/75 ━━━━━━━━━━━━━━━━━━━━ 22s 291ms/step - accuracy: 0.8841 - loss: 0.3598 - val_accuracy: 0.8905 - val_loss: 0.3482


In [37]:
print(X_train.shape) 
print(y_train.shape)  


(4745, 172)
(4745,)


In [36]:
y_train = np.array(y_train)

print(X_train.shape) 
print(y_train.shape) 


(4745, 172)
(4745,)


In [39]:
y_train = np.array(y_train).flatten()
y_test = np.array(y_test).flatten()

print(y_train.shape) 
print(y_test.shape)  


(4745,)
(1187,)


In [41]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.8893 - loss: 0.3510
Test Accuracy: 89.05%
